In [1]:
"""ATE: Automatic Term Extraction
"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import MeCabTokenizer


In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [3]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [6]:
print("\n".join(docs[2023][:15]))

生体医工学ウェブ辞典（第二分冊）
医療機関等への建築ガイドライン浸透のための課題と今後の展望
メーカーから見た医用テレメータ設置・運用に関する現状と確認事項
医用テレメータの概要と医療現場での現状と希望
医療機関における電波利用機器に配慮した建築ガイドライン・同解説－医用テレメータ編－のポイント
生体医工学×医療者教育―効率的な知識・技術発展のためにー
生体医工学×知のデザイン －臨床研究における学際の必要性－
生体医工学×医工連携～自身の開発経験談～
生体医工学×医療機器管理のエビデンス創設を目指して
スマートコントラクトを用いたトラストデータ流通システム
聴診音解析システムを活用した医療DXへの挑戦
看護理工学的思考によるリアルワールドデータの活用
皮膚感覚メカニズム解明のためのセンシング・ディスプレイ技術
神経オルガノイドを用いた特殊培養デバイスによる疼痛評価システム
次世代網膜刺激型人工視覚デバイスの開発


In [12]:
year = 2019

tok = MeCabTokenizer() 
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題',"および", "評価"]
ignores1 = ['型']
ignores2 = ['中','的','時','システム','用']

compound_nouns = []
for y in years:
    for sentence in docs[y]:
        if sentence.isascii():
            continue 
    
        tokens = tok.iter_token(sentence)
        nouns = tok.extract_nouns(tokens)
        # print(nouns)
        nouns = tok.filter_nouns(nouns, n=1, stopwords=ignores, stopwords_begin=ignores1, stopwords_end=ignores2)
        # print(nouns)
        if len(nouns) > 0 :
            compound_nouns.extend(nouns)

lr = LRValue()
# lr_values = lr.fit_transform(compound_nouns)
lr_values = lr.fit(compound_nouns).transform(compound_nouns, mode=3)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)
list(data_collection)[:50]

[('性', 98.67623827447011),
 ('計測', 78.689262291624),
 ('画像 計測', 70.22196244621308),
 ('機能 計測', 67.30984959259138),
 ('細胞', 64.89992295835181),
 ('循環 型', 64.06739834234645),
 ('画像', 62.66578013557319),
 ('細胞 機能', 61.12839229772927),
 ('計測 解析', 60.807549803733956),
 ('細胞 運動', 60.80310614619487),
 ('細胞 モデル', 59.9759855865452),
 ('計測 光', 59.83431497435334),
 ('光 計測', 59.83431497435334),
 ('生体 画像 計測', 59.265223258486486),
 ('超音波 画像 計測', 58.778651212904556),
 ('脳 機能 計測', 57.8393254104299),
 ('生体 計測', 57.634782933863946),
 ('生体 機能 計測', 57.615194192879436),
 ('機能', 57.57603668193913),
 ('運動 機能', 57.26965389173499),
 ('運動', 56.96490147450446),
 ('呼吸 性', 55.57926656262697),
 ('モデル', 55.42562584220407),
 ('画像 刺激', 55.233467904674896),
 ('刺激 画像', 55.233467904674896),
 ('細胞 解析', 55.22323674120181),
 ('細胞 組織', 54.92063227471158),
 ('光 脳 機能 計測', 54.47079420302772),
 ('自動 計測', 52.713803130502775),
 ('細胞 化', 52.53899152565192),
 ('呼吸 機能 計測', 52.15029378675262),
 ('血管 細胞', 52.09781905153768),
 ('生体 画像',

In [18]:
terms = lr_values
freq = lr.compound_noun
terms.sort(key=lambda z: (len(z[0].split()), len(z[0]),-z[1]), reverse=False)
n, m = 2100, 20
for i, v in terms[n:n+m]:
    print(i, v, freq[i])

補助人工心臓 5.916079783099616 2
コーティング 4.47213595499958 2
ダイナミクス 4.242640687119285 5
視覚誘発電位 4.0 6
ガイドライン 4.0 1
システム開発 3.7416573867739413 1
データベース 3.4641016151377544 1
ビッグデータ 3.4641016151377544 3
トモグラフィ 3.4641016151377544 2
コンピュータ 3.4641016151377544 1
脈波伝播速度 3.0 1
コントロール 2.8284271247461903 1
ソフトウェア 2.6457513110645907 1
加速度センサ 2.449489742783178 9
スケーリング 2.449489742783178 1
独立成分分析 2.449489742783178 3
間葉系幹細胞 2.449489742783178 5
リモデリング 2.449489742783178 1
バリアフリー 2.449489742783178 1
臨床工学技士 2.449489742783178 6
